# **Chapter 8: Avoiding Hallucinations**

---
**Lesson:**

Some bad news: Claude sometimes "hallucinates" and makes claims that are untrue or unjustified. The good news: there are techniques you can use to minimize hallucinations.				
Below, we'll go over a few of these techniques, namely:

- Giving Claude the option to say it doesn't know the answer to a question 
- Asking Claude to find the evidence before answering 
---

First we will setup our dependencies

In [2]:
%%capture
#Install dependencies
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install --quiet langchain==0.0.304

#Import libraries, and set up Bedrock client
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

#Set up model inference modifiers, and ensure Claude v2 is being called from Bedrock 
from langchain.llms.bedrock import Bedrock
from langchain import PromptTemplate

inference_modifier = {
    "max_tokens_to_sample": 4096,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}

textgen_llm = Bedrock(
    model_id="anthropic.claude-v2:1",
    client=boto3_bedrock,
    model_kwargs=inference_modifier,
)

# **Examples:**

**Example 8.1 - Hungry Hippos**

Here is a question about general factual knowledge to which Claude hallucinates an answer because it's trying to be as helpful as possible.

In [3]:
response = textgen_llm("""

Human: Who is the heaviest hippo ever recorded?  

Assistant:""")

print_ww(response)

 Based on my research, the heaviest hippo on record was a male named Humphrey. He lived in a zoo in
the South of England and reached a weight of at least 4,500 pounds (over 2,000 kilograms) before he
died in 2018 at an estimated age of 50.

Humphrey was exceptionally large, even for a hippo. Male hippos on average weigh between 3,000-4,000
pounds in the wild. Humphrey likely was able to achieve his massive weight from constant access to
plentiful food provided in his captive environment. Records of the largest hippos are mostly tracked
among captive animals since they can be regularly weighed. It's possible even heavier wild hippos
may exist, but their weights would be harder to verify. Regardless, at over 2 tons, Humphrey appears
to be the verifiable record holder for the heaviest hippo ever documented.


**Example 8.2 - Hungry Hippos 2**

A solution we can try here is to "give Claude an out" — tell Claude that it's OK for it to decline to answer, or to only answer if it actually knows the answer with certainty.

In [4]:
response = textgen_llm("""

Human: Who is the heaviest hippo ever recorded? Only answer if you know the answer with certainty  

Assistant:""")

print_ww(response)

 Unfortunately, I do not have enough information to definitively state the heaviest hippo ever
recorded. Hippo weights can vary substantially, and tracking the heaviest individuals over time
would require very specific dataseta that I do not have access to. Without a credible source to
reference, I cannot provide a certain answer to this question. I'd be happy to search for more
information if you have any details that could help narrow it down. Please let me know if you have a
specific source you'd like me to consult.


# **Exercise**

The following exercises will need you to manipulate the prompt to get the desired output

**Exercise 8.1 - NBA Hallucination**

Adapt the prompt in the cell below to fix Claude's hallucination issue by giving Claude an out. (Kawhi Leonard has only won two NBA championships.) 

In [8]:
response = textgen_llm("""

Human: In what year did Kawhi Leonard win his third NBA championship? Think step-by-step.  

Assistant:""")

print_ww(response)

 Unfortunately, I do not have enough context to definitively state what year Kawhi Leonard won his
third NBA championship. However, I can walk through a logical step-by-step process to try to
determine it:

1) Kawhi Leonard has won 2 NBA championships so far in his career - in 2014 with the San Antonio
Spurs and in 2019 with the Toronto Raptors.

2) If he were to win a 3rd championship, it would have to be in a year after 2019, since that was
his 2nd championship.

3) As of 2022, Kawhi Leonard has not won another championship since 2019.

4) Therefore, without additional information, I cannot conclude what year Leonard won a 3rd NBA
championship, since the available evidence suggests he has only won 2 championships so far in 2014
and 2019.

The key steps are recognizing he has won 2 titles, so a 3rd would have to come after 2019, then
confirming he has not won since 2019 as of 2022, so there is no evidence for me to state a year he
won a 3rd championship. I would need more contextual i

# Chapter 8 - END.